In [1]:
import pdb


import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import theano
import pandas as pd
print(f"Running on PyMC3 v{pm.__version__}")
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

Running on PyMC3 v3.11.2


In [2]:
samples = 2500
tune = 2500
# with 1000 there are some convergence issues(r hat) especially with cluster 2

In [3]:
# data = pd.read_csv('test_lme_data.csv')
data_clust1 = pd.read_csv('FaNoOcc_NeNoOcc_insta_abs_detrend_lme_smile_cluster_beta_data_IBS.csv')

# data_clust1.head()
# data = data.iloc[0:7000,]
data_clust1.condition = data_clust1.condition_Smile_auto.str.split("_",n=1, expand = True)[1]
data_clust1.condition = data_clust1.condition.astype('int')

<ipython-input-3-c0790adf876a>:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data_clust1.condition = data_clust1.condition_Smile_auto.str.split("_",n=1, expand = True)[1]


In [4]:
n_dyads = len(data_clust1.Dyad_no_Gaze_nose_dist.unique())
dyad_idx = np.array(data_clust1.Dyad_no_Gaze_nose_dist.str.split("_",n=1, expand = True)[1],dtype='int64')-1

with pm.Model() as hierarchical_model_clust_1_null:
    # Hyperpriors for group nodes
    mu_intercept = pm.Normal("mu_intercept", mu=0.0, sigma=100)
    sigma_intercept = pm.HalfNormal("sigma_intercept", 5.0)
    mu_eye_gaze = pm.Normal("mu_eye_gaze", mu=0.0, sigma=100)
    sigma_eye_gaze = pm.HalfNormal("sigma_eye_gaze", 5.0)    
  
    mu_mouth_size = pm.Normal("mu_mouth_size", mu=0.0, sigma=100)
    sigma_mouth_size = pm.HalfNormal("sigma_mouth_size", 5.0)  
    
    mu_mov = pm.Normal("mu_mov", mu=0.0, sigma=100)
    sigma_mov = pm.HalfNormal("sigma_mov", 5.0)     
    
    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    intercept = pm.Normal("intercept", mu=mu_intercept, sigma=sigma_intercept, shape=n_dyads)
    # Intercept for each county, distributed around group mean mu_a
    eye_gaze = pm.Normal("eye_gaze", mu=mu_eye_gaze, sigma=sigma_eye_gaze, shape=n_dyads)
    mouth_size = pm.Normal("mouth_size", mu=mu_mouth_size, sigma=sigma_mouth_size, shape=n_dyads)
    mov = pm.Normal("mov", mu=mu_mov, sigma=sigma_mov, shape=n_dyads)
    # Model error
    eps = pm.HalfCauchy("eps", 5.0)
    #pdb.set_trace()
#     feature_est = intercept[dyad_idx]  + condition[dyad_idx] * data.condition.values +  eye_gaze[dyad_idx] * data.eye_gaze_distance_joint.values + mouth_size[dyad_idx] * data.mouth_size_joint.values + e[dyad_idx] * data.ALL.values
    feature_est = intercept[dyad_idx] 

    # Data likelihood
    feature_like = pm.Normal("feature_like", mu=feature_est, sigma=eps, observed=data_clust1.chan_freq_data)
#     hierarchical_trace_clust_1_null = pm.sample(5000, tune=5000, target_accept=0.99)
#     hierarchical_trace_clust_1_null = pm.sample(samples, tune=tune, target_accept=0.99)

In [5]:
# trace 18
# hierarchical_trace_clust_1_null_save = pm.save_trace(hierarchical_trace_clust_1_null,directory='.null_model_clust_1_abs_glm.trace')    

In [6]:
with hierarchical_model_clust_1_null:
    hierarchical_trace_clust_1_null = pm.load_trace(".null_model_clust_1_abs_glm_2500.trace")

In [7]:
n_dyads = len(data_clust1.Dyad_no_Gaze_nose_dist.unique())
dyad_idx = np.array(data_clust1.Dyad_no_Gaze_nose_dist.str.split("_",n=1, expand = True)[1],dtype='int64')-1

with pm.Model() as hierarchical_model_clust_1:
    # Hyperpriors for group nodes
    mu_intercept = pm.Normal("mu_intercept", mu=0.0, sigma=100)
    sigma_intercept = pm.HalfNormal("sigma_intercept", 5.0)
    mu_eye_gaze = pm.Normal("mu_eye_gaze", mu=0.0, sigma=100)
    sigma_eye_gaze = pm.HalfNormal("sigma_eye_gaze", 5.0)    
  
    mu_mouth_size = pm.Normal("mu_mouth_size", mu=0.0, sigma=100)
    sigma_mouth_size = pm.HalfNormal("sigma_mouth_size", 5.0)  
    
    mu_mov = pm.Normal("mu_mov", mu=0.0, sigma=100)
    sigma_mov = pm.HalfNormal("sigma_mov", 5.0)     
    
    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    intercept = pm.Normal("intercept", mu=mu_intercept, sigma=sigma_intercept, shape=n_dyads)
    # Intercept for each county, distributed around group mean mu_a
    eye_gaze = pm.Normal("eye_gaze", mu=mu_eye_gaze, sigma=sigma_eye_gaze, shape=n_dyads)
    mouth_size = pm.Normal("mouth_size", mu=mu_mouth_size, sigma=sigma_mouth_size, shape=n_dyads)
    mov = pm.Normal("mov", mu=mu_mov, sigma=sigma_mov, shape=n_dyads)
    # Model error
    eps = pm.HalfCauchy("eps", 5.0)
    #pdb.set_trace()
#     feature_est = intercept[dyad_idx]  + condition[dyad_idx] * data.condition.values +  eye_gaze[dyad_idx] * data.eye_gaze_distance_joint.values + mouth_size[dyad_idx] * data.mouth_size_joint.values + e[dyad_idx] * data.ALL.values
    feature_est = intercept[dyad_idx]  +  eye_gaze[dyad_idx] * data_clust1.eye_gaze_distance_joint.values + mouth_size[dyad_idx] * data_clust1.mouth_size_joint.values +  mov[dyad_idx] * data_clust1.ALL_joint.values

    # Data likelihood
    feature_like = pm.Normal("feature_like", mu=feature_est, sigma=eps, observed=data_clust1.chan_freq_data)
#     hierarchical_trace_clust_1 = pm.sample(samples, tune=tune, target_accept=0.99)

In [8]:
# hierarchical_trace_clust_1_save = pm.save_trace(hierarchical_trace_clust_1,directory='.model_clust_1_abs_glm.trace')

In [9]:
with hierarchical_model_clust_1:
    hierarchical_trace_clust_1 = pm.load_trace(".model_clust_1_abs_glm_2500.trace")

In [10]:
hierarchical_trace_clust_1_null_az = az.from_pymc3(hierarchical_trace_clust_1_null)
hierarchical_trace_clust_1_az = az.from_pymc3(hierarchical_trace_clust_1)
compare_dict = {"null": hierarchical_trace_clust_1_null_az, "clust_1": hierarchical_trace_clust_1_az}
az.compare(compare_dict)

C:\Users\SuperLabPC\anaconda3\envs\pymc\lib\site-packages\arviz\data\io_pymc3.py:96: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(
C:\Users\SuperLabPC\anaconda3\envs\pymc\lib\site-packages\arviz\data\io_pymc3.py:96: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(
C:\Users\SuperLabPC\anaconda3\envs\pymc\lib\site-packages\arviz\stats\stats.py:145: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  warnings.warn(
C:\Users\SuperLabPC\anaconda3\envs\pymc\lib\site-packages\arviz\stats\stats.py:876: Runtim

,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
clust_1,0,-224617.119326,27.412576,0.000000,0.536965,283.054908,0.0000,False,log
null,1,-224617.663217,2.678311,0.543892,0.463035,283.106982,3.8355,False,log


In [11]:
print(np.sum(hierarchical_trace_clust_1['mu_eye_gaze']<0)/len(hierarchical_trace_clust_1['mu_eye_gaze']))
print(np.sum(hierarchical_trace_clust_1['mu_mouth_size']<0)/len(hierarchical_trace_clust_1['mu_mouth_size']))
print(np.sum(hierarchical_trace_clust_1['mu_mov']<0)/len(hierarchical_trace_clust_1['mu_mov']))


0.0421
0.5708
0.336


In [12]:
# az.rhat(hierarchical_trace_clust_1,var_names=["mu_eye_gaze", "mu_mouth_size","mu_mov"])

In [13]:
# import operator
# from functools import reduce
# np.sum(reduce(operator.and_,[hierarchical_trace_clust_1['mu_eye_gaze']<0.01, \
#                             hierarchical_trace_clust_1['mu_eye_gaze']>-0.01]))/\
# len(hierarchical_trace_clust_1['mu_eye_gaze'])

In [14]:
# az.plot_posterior(hierarchical_trace_clust_1, var_names=["mu_eye_gaze","mu_mouth_size", "mu_mov"],ref_val=0,linewidth=2)

In [15]:
# root_dir = "E:\\Projects\\IBS\\Results\\EEG\\Brain_behavior_glm_power_freqwise\\no_aggressive_CAR_ASR_10_ICA_appended_trials\\figures\\"
# fig = plt.figure()
# ax = fig.add_subplot(111)
# a = az.plot_posterior(hierarchical_trace_clust_1, var_names=["mu_eye_gaze"],ref_val=0,linewidth=2,ax =ax,hdi_prob=0.90)#,bw=0.002
# fig.savefig(root_dir + "GLM_cluster_1_eye_insta_abs_detrend_smile_cor7.eps",format='eps')
# fig = plt.figure()
# ax = fig.add_subplot(111)
# a = az.plot_posterior(hierarchical_trace_clust_1, var_names=["mu_mouth_size"],ref_val=0,linewidth=2,ax =ax,hdi_prob=0.90)
# fig.savefig(root_dir + "GLM_cluster_1_mouth_insta_abs_detrend_smile_cor7.eps",format='eps')
# fig = plt.figure()
# ax = fig.add_subplot(111)
# a = az.plot_posterior(hierarchical_trace_clust_1, var_names=["mu_mov"],ref_val=0,linewidth=2,ax =ax,hdi_prob=0.90)
# fig.savefig(root_dir + "GLM_cluster_1_mov_insta_abs_detrend_smile_cor7.eps",format='eps')


# a = az.plot_posterior(hierarchical_trace_clust_1, var_names=["mu_eye_gaze","mu_mouth_size", "mu_mov"],ref_val=0,linewidth=2,ax =ax)

In [16]:
# import seaborn as sns

In [17]:
# sns.distplot(hierarchical_trace_clust_1['mu_mouth_size'])

In [18]:
# bfmi = np.max(pm.stats.bfmi(trace))
# max_gr = max(np.max(gr_stats) for gr_stats in pm.stats.rhat(trace).values()).values


# (
#     pm.energyplot(trace, legend=False, figsize=(6, 4)).set_title(
#         f"BFMI = {bfmi}\nGelman-Rubin = {max_gr}"
#     )
# )

In [19]:
%reset -f

In [20]:
import pdb


import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import theano
import pandas as pd
print(f"Running on PyMC3 v{pm.__version__}")
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

Running on PyMC3 v3.11.2


In [21]:
samples = 2500
tune = 2500
# with 1000 there are some convergence issues(r hat) especially with cluster 2

In [22]:
# data = pd.read_csv('test_lme_data.csv')
data_clust2 = pd.read_csv('FaNoOcc_NeNoOcc_insta_abs_detrend_lme_smile_cluster_gamma_data_IBS.csv')
data_clust2.head()
# data = data.iloc[0:7000,]
data_clust2.condition = data_clust2.condition_Smile_auto.str.split("_",n=1, expand = True)[1]
data_clust2.condition = data_clust2.condition.astype('int')

<ipython-input-22-eec3f94ffd65>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data_clust2.condition = data_clust2.condition_Smile_auto.str.split("_",n=1, expand = True)[1]


In [23]:
n_dyads = len(data_clust2.Dyad_no_Gaze_nose_dist.unique())
dyad_idx = np.array(data_clust2.Dyad_no_Gaze_nose_dist.str.split("_",n=1, expand = True)[1],dtype='int64')-1

with pm.Model() as hierarchical_model_clust_2_null:
    # Hyperpriors for group nodes
    mu_intercept = pm.Normal("mu_intercept", mu=0.0, sigma=100)
    sigma_intercept = pm.HalfNormal("sigma_intercept", 5.0)
    mu_eye_gaze = pm.Normal("mu_eye_gaze", mu=0.0, sigma=100)
    sigma_eye_gaze = pm.HalfNormal("sigma_eye_gaze", 5.0)    
  
    mu_mouth_size = pm.Normal("mu_mouth_size", mu=0.0, sigma=100)
    sigma_mouth_size = pm.HalfNormal("sigma_mouth_size", 5.0)  
    
    mu_mov = pm.Normal("mu_mov", mu=0.0, sigma=100)
    sigma_mov = pm.HalfNormal("sigma_mov", 5.0)     
    
    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    intercept = pm.Normal("intercept", mu=mu_intercept, sigma=sigma_intercept, shape=n_dyads)
    # Intercept for each county, distributed around group mean mu_a
    eye_gaze = pm.Normal("eye_gaze", mu=mu_eye_gaze, sigma=sigma_eye_gaze, shape=n_dyads)
    mouth_size = pm.Normal("mouth_size", mu=mu_mouth_size, sigma=sigma_mouth_size, shape=n_dyads)
    mov = pm.Normal("mov", mu=mu_mov, sigma=sigma_mov, shape=n_dyads)
    # Model error
    eps = pm.HalfCauchy("eps", 5.0)
    #pdb.set_trace()
#     feature_est = intercept[dyad_idx]  + condition[dyad_idx] * data.condition.values +  eye_gaze[dyad_idx] * data.eye_gaze_distance_joint.values + mouth_size[dyad_idx] * data.mouth_size_joint.values + e[dyad_idx] * data.ALL.values
    feature_est = intercept[dyad_idx]  

    # Data likelihood
    feature_like = pm.Normal("feature_like", mu=feature_est, sigma=eps, observed=data_clust2.chan_freq_data)
#     hierarchical_trace_clust_2_null = pm.sample(samples, tune=tune, target_accept=0.99)

In [24]:
# trace 18
# hierarchical_trace_clust_2_null_save = pm.save_trace(hierarchical_trace_clust_2_null,directory='.null_model_clust_2_abs_glm.trace')

In [25]:
with hierarchical_model_clust_2_null:
    hierarchical_trace_clust_2_null = pm.load_trace(".null_model_clust_2_abs_glm_2500.trace")

In [26]:
n_dyads = len(data_clust2.Dyad_no_Gaze_nose_dist.unique())
dyad_idx = np.array(data_clust2.Dyad_no_Gaze_nose_dist.str.split("_",n=1, expand = True)[1],dtype='int64')-1

with pm.Model() as hierarchical_model_clust_2:
    # Hyperpriors for group nodes
    mu_intercept = pm.Normal("mu_intercept", mu=0.0, sigma=100)
    sigma_intercept = pm.HalfNormal("sigma_intercept", 5.0)
    mu_eye_gaze = pm.Normal("mu_eye_gaze", mu=0.0, sigma=100)
    sigma_eye_gaze = pm.HalfNormal("sigma_eye_gaze", 5.0)    
  
    mu_mouth_size = pm.Normal("mu_mouth_size", mu=0.0, sigma=100)
    sigma_mouth_size = pm.HalfNormal("sigma_mouth_size", 5.0)  
    
    mu_mov = pm.Normal("mu_mov", mu=0.0, sigma=100)
    sigma_mov = pm.HalfNormal("sigma_mov", 5.0)     
    
    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    intercept = pm.Normal("intercept", mu=mu_intercept, sigma=sigma_intercept, shape=n_dyads)
    # Intercept for each county, distributed around group mean mu_a
    eye_gaze = pm.Normal("eye_gaze", mu=mu_eye_gaze, sigma=sigma_eye_gaze, shape=n_dyads)
    mouth_size = pm.Normal("mouth_size", mu=mu_mouth_size, sigma=sigma_mouth_size, shape=n_dyads)
    mov = pm.Normal("mov", mu=mu_mov, sigma=sigma_mov, shape=n_dyads)
    # Model error
    eps = pm.HalfCauchy("eps", 5.0)
    #pdb.set_trace()
#     feature_est = intercept[dyad_idx]  + condition[dyad_idx] * data.condition.values +  eye_gaze[dyad_idx] * data.eye_gaze_distance_joint.values + mouth_size[dyad_idx] * data.mouth_size_joint.values + e[dyad_idx] * data.ALL.values
    feature_est = intercept[dyad_idx]  +  eye_gaze[dyad_idx] * data_clust2.eye_gaze_distance_joint.values + mouth_size[dyad_idx] * data_clust2.mouth_size_joint.values +  mov[dyad_idx] * data_clust2.ALL_joint.values

    # Data likelihood
    feature_like = pm.Normal("feature_like", mu=feature_est, sigma=eps, observed=data_clust2.chan_freq_data)
#     hierarchical_trace_clust_2 = pm.sample(samples, tune=tune, target_accept=0.99)

In [27]:
# trace 19
# hierarchical_trace_clust_2_save = pm.save_trace(hierarchical_trace_clust_2,directory='.model_clust_2_abs_glm.trace')

In [28]:
with hierarchical_model_clust_2:
    hierarchical_trace_clust_2 = pm.load_trace(".model_clust_2_abs_glm_2500.trace")

In [29]:
print(np.sum(hierarchical_trace_clust_2['mu_eye_gaze']<0)/len(hierarchical_trace_clust_2['mu_eye_gaze']))
print(np.sum(hierarchical_trace_clust_2['mu_mouth_size']<0)/len(hierarchical_trace_clust_2['mu_mouth_size']))
print(np.sum(hierarchical_trace_clust_2['mu_mov']<0)/len(hierarchical_trace_clust_2['mu_mov']))


0.5434
0.0
0.0002


In [30]:
# az.plot_posterior(hierarchical_trace_clust_2, var_names=["mu_eye_gaze","mu_mouth_size", "mu_mov"],ref_val=0,linewidth=2)

In [31]:
# root_dir = "E:\\Projects\\IBS\\Results\\EEG\\Brain_behavior_glm_power_freqwise\\no_aggressive_CAR_ASR_10_ICA_appended_trials\\figures\\"
# fig = plt.figure()
# ax = fig.add_subplot(111)
# a = az.plot_posterior(hierarchical_trace_clust_2, var_names=["mu_eye_gaze"],ref_val=0,linewidth=2,ax =ax,hdi_prob=0.90)
# fig.savefig(root_dir + "GLM_cluster_2_eye_insta_abs_detrend_smile_cor7.eps",format='eps')
# fig = plt.figure()
# ax = fig.add_subplot(111)
# a = az.plot_posterior(hierarchical_trace_clust_2, var_names=["mu_mouth_size"],ref_val=0,linewidth=2,ax =ax,hdi_prob=0.90)
# fig.savefig(root_dir + "GLM_cluster_2_mouth_insta_abs_detrend_smile_cor7.eps",format='eps')
# fig = plt.figure()
# ax = fig.add_subplot(111)
# a = az.plot_posterior(hierarchical_trace_clust_2, var_names=["mu_mov"],ref_val=0,linewidth=2,ax =ax,hdi_prob=0.90)
# fig.savefig(root_dir + "GLM_cluster_2_mov_insta_abs_detrend_smile_cor7.eps",format='eps')



In [32]:
# df_trace = pm.trace_to_dataframe(hierarchical_trace_clust_2)

In [33]:
# az.rhat(hierarchical_trace_clust_1)

In [34]:
# az.rhat(hierarchical_trace_clust_2)

In [35]:
# hierarchical_trace_clust_1_null = az.from_pymc3(hierarchical_trace_clust_1_null)
# hierarchical_trace_clust_1 = az.from_pymc3(hierarchical_trace_clust_1)

# compare_dict = {"null": hierarchical_trace_clust_1_null, "clust_1": hierarchical_trace_clust_1}
# az.compare(compare_dict)

In [36]:
hierarchical_trace_clust_2_null_az = az.from_pymc3(hierarchical_trace_clust_2_null)
hierarchical_trace_clust_2_az = az.from_pymc3(hierarchical_trace_clust_2)

compare_dict = {"null": hierarchical_trace_clust_2_null_az, "clust_1": hierarchical_trace_clust_2_az}
az.compare(compare_dict)

C:\Users\SuperLabPC\anaconda3\envs\pymc\lib\site-packages\arviz\data\io_pymc3.py:96: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(
C:\Users\SuperLabPC\anaconda3\envs\pymc\lib\site-packages\arviz\stats\stats.py:145: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  warnings.warn(
C:\Users\SuperLabPC\anaconda3\envs\pymc\lib\site-packages\arviz\stats\stats.py:876: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)
C:\Users\SuperLabPC\anaconda3\envs\pymc\lib\site-packages\numpy\core\_methods.py:47: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
clust_1,0,-223359.724709,58.501048,0.000000,1.0,327.984708,0.000000,False,log
null,1,-223632.614923,3.564646,272.890215,0.0,328.334685,23.187599,False,log


In [37]:
# az.rhat(hierarchical_trace_clust_2,var_names=["mu_eye_gaze", "mu_mouth_size","mu_mov"])

In [38]:
%reset -f

In [39]:
import pdb


import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import theano
import pandas as pd
print(f"Running on PyMC3 v{pm.__version__}")
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

Running on PyMC3 v3.11.2


In [40]:
# data = pd.read_csv('test_lme_data.csv')FaNoOcc_NeNoOcc_insta_corr_avg_freqwise_lme_smile_cluster_3_data_IBS
data_clust3 = pd.read_csv('FaNoOcc_NeNoOcc_insta_abs_detrend_lme_smile_cluster_alpha_data_IBS.csv')

data_clust3.head()
# data = data.iloc[0:7000,]
data_clust3.condition = data_clust3.condition_Smile_auto.str.split("_",n=1, expand = True)[1]
data_clust3.condition = data_clust3.condition.astype('int')

<ipython-input-40-1876bfe8747f>:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data_clust3.condition = data_clust3.condition_Smile_auto.str.split("_",n=1, expand = True)[1]


In [41]:
n_dyads = len(data_clust3.Dyad_no_Gaze_nose_dist.unique())
dyad_idx = np.array(data_clust3.Dyad_no_Gaze_nose_dist.str.split("_",n=1, expand = True)[1],dtype='int64')-1

with pm.Model() as hierarchical_model_clust_3_null:
    # Hyperpriors for group nodes
    mu_intercept = pm.Normal("mu_intercept", mu=0.0, sigma=100)
    sigma_intercept = pm.HalfNormal("sigma_intercept", 5.0)
    mu_eye_gaze = pm.Normal("mu_eye_gaze", mu=0.0, sigma=100)
    sigma_eye_gaze = pm.HalfNormal("sigma_eye_gaze", 5.0)    
  
    mu_mouth_size = pm.Normal("mu_mouth_size", mu=0.0, sigma=100)
    sigma_mouth_size = pm.HalfNormal("sigma_mouth_size", 5.0)  
    
    mu_mov = pm.Normal("mu_mov", mu=0.0, sigma=100)
    sigma_mov = pm.HalfNormal("sigma_mov", 5.0)     
    
    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    intercept = pm.Normal("intercept", mu=mu_intercept, sigma=sigma_intercept, shape=n_dyads)
    # Intercept for each county, distributed around group mean mu_a
    eye_gaze = pm.Normal("eye_gaze", mu=mu_eye_gaze, sigma=sigma_eye_gaze, shape=n_dyads)
    mouth_size = pm.Normal("mouth_size", mu=mu_mouth_size, sigma=sigma_mouth_size, shape=n_dyads)
    mov = pm.Normal("mov", mu=mu_mov, sigma=sigma_mov, shape=n_dyads)
    # Model error
    eps = pm.HalfCauchy("eps", 5.0)
    #pdb.set_trace()
#     feature_est = intercept[dyad_idx]  + condition[dyad_idx] * data.condition.values +  eye_gaze[dyad_idx] * data.eye_gaze_distance_joint.values + mouth_size[dyad_idx] * data.mouth_size_joint.values + e[dyad_idx] * data.ALL.values
    feature_est = intercept[dyad_idx]  

    # Data likelihood
    feature_like = pm.Normal("feature_like", mu=feature_est, sigma=eps, observed=data_clust3.chan_freq_data)
#     hierarchical_trace_clust_3_null = pm.sample(samples, tune=tune, target_accept=0.99)

In [42]:
with hierarchical_model_clust_3_null:
    hierarchical_trace_clust_3_null = pm.load_trace(".null_model_clust_3_abs_glm_2500.trace")

In [43]:
n_dyads = len(data_clust3.Dyad_no_Gaze_nose_dist.unique())
dyad_idx = np.array(data_clust3.Dyad_no_Gaze_nose_dist.str.split("_",n=1, expand = True)[1],dtype='int64')-1

with pm.Model() as hierarchical_model_clust_3:
    # Hyperpriors for group nodes
    mu_intercept = pm.Normal("mu_intercept", mu=0.0, sigma=100)
    sigma_intercept = pm.HalfNormal("sigma_intercept", 5.0)
    mu_eye_gaze = pm.Normal("mu_eye_gaze", mu=0.0, sigma=100)
    sigma_eye_gaze = pm.HalfNormal("sigma_eye_gaze", 5.0)    
  
    mu_mouth_size = pm.Normal("mu_mouth_size", mu=0.0, sigma=100)
    sigma_mouth_size = pm.HalfNormal("sigma_mouth_size", 5.0)  
    
    mu_mov = pm.Normal("mu_mov", mu=0.0, sigma=100)
    sigma_mov = pm.HalfNormal("sigma_mov", 5.0)     
    
    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    intercept = pm.Normal("intercept", mu=mu_intercept, sigma=sigma_intercept, shape=n_dyads)
    # Intercept for each county, distributed around group mean mu_a
    eye_gaze = pm.Normal("eye_gaze", mu=mu_eye_gaze, sigma=sigma_eye_gaze, shape=n_dyads)
    mouth_size = pm.Normal("mouth_size", mu=mu_mouth_size, sigma=sigma_mouth_size, shape=n_dyads)
    mov = pm.Normal("mov", mu=mu_mov, sigma=sigma_mov, shape=n_dyads)
    # Model error
    eps = pm.HalfCauchy("eps", 5.0)
    #pdb.set_trace()
#     feature_est = intercept[dyad_idx]  + condition[dyad_idx] * data.condition.values +  eye_gaze[dyad_idx] * data.eye_gaze_distance_joint.values + mouth_size[dyad_idx] * data.mouth_size_joint.values + e[dyad_idx] * data.ALL.values
    feature_est = intercept[dyad_idx]  +  eye_gaze[dyad_idx] * data_clust3.eye_gaze_distance_joint.values + mouth_size[dyad_idx] * data_clust3.mouth_size_joint.values +  mov[dyad_idx] * data_clust3.ALL_joint.values

    # Data likelihood
    feature_like = pm.Normal("feature_like", mu=feature_est, sigma=eps, observed=data_clust3.chan_freq_data)
#     hierarchical_trace_clust_3 = pm.sample(samples, tune=tune, target_accept=0.99)

In [44]:
with hierarchical_model_clust_3:
    hierarchical_trace_clust_3 = pm.load_trace(".model_clust_3_abs_glm_2500.trace")

In [45]:
print(np.sum(hierarchical_trace_clust_3['mu_eye_gaze']<0)/len(hierarchical_trace_clust_3['mu_eye_gaze']))
print(np.sum(hierarchical_trace_clust_3['mu_mouth_size']<0)/len(hierarchical_trace_clust_3['mu_mouth_size']))
print(np.sum(hierarchical_trace_clust_3['mu_mov']<0)/len(hierarchical_trace_clust_3['mu_mov']))


0.2676
0.572
0.8422


In [46]:
hierarchical_trace_clust_3_null_az = az.from_pymc3(hierarchical_trace_clust_3_null)
hierarchical_trace_clust_3_az = az.from_pymc3(hierarchical_trace_clust_3)
compare_dict = {"null": hierarchical_trace_clust_3_null_az, "clust_1": hierarchical_trace_clust_3_az}
az.compare(compare_dict)

C:\Users\SuperLabPC\anaconda3\envs\pymc\lib\site-packages\arviz\data\io_pymc3.py:96: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(
C:\Users\SuperLabPC\anaconda3\envs\pymc\lib\site-packages\arviz\stats\stats.py:145: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  warnings.warn(
C:\Users\SuperLabPC\anaconda3\envs\pymc\lib\site-packages\arviz\stats\stats.py:876: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)
C:\Users\SuperLabPC\anaconda3\envs\pymc\lib\site-packages\numpy\core\_methods.py:47: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
clust_1,0,-224594.525923,38.520849,0.000000,0.888303,259.726523,0.000000,False,log
null,1,-224615.002957,3.308509,20.477034,0.111697,259.729563,7.261084,False,log


In [47]:
# az.rhat(hierarchical_trace_clust_3,var_names=["mu_eye_gaze", "mu_mouth_size","mu_mov"])

In [48]:
# root_dir = "E:\\Projects\\IBS\\Results\\EEG\\Brain_behavior_glm_power_freqwise\\no_aggressive_CAR_ASR_10_ICA_appended_trials\\figures\\"
# fig = plt.figure()
# ax = fig.add_subplot(111)
# a = az.plot_posterior(hierarchical_trace_clust_3, var_names=["mu_eye_gaze"],ref_val=0,linewidth=2,ax =ax,hdi_prob=0.90)
# fig.savefig(root_dir + "GLM_cluster_3_eye_insta_abs_detrend_smile_cor7.eps",format='eps')
# fig = plt.figure()
# ax = fig.add_subplot(111)
# a = az.plot_posterior(hierarchical_trace_clust_3, var_names=["mu_mouth_size"],ref_val=0,linewidth=2,ax =ax,hdi_prob=0.90)
# fig.savefig(root_dir + "GLM_cluster_3_mouth_insta_abs_detrend_smile_cor7.eps",format='eps')
# fig = plt.figure()
# ax = fig.add_subplot(111)
# a = az.plot_posterior(hierarchical_trace_clust_3, var_names=["mu_mov"],ref_val=0,linewidth=2,ax =ax,hdi_prob=0.90)
# fig.savefig(root_dir + "GLM_cluster_3_mov_insta_abs_detrend_smile_cor7.eps",format='eps')



In [49]:
# az.plot_posterior(hierarchical_trace_clust_3, var_names=["mu_eye_gaze"],ref_val=0,linewidth=2,hdi_prob=0.90,bw=0.0001)

In [50]:
# pm.plot_trace(hierarchical_trace_clust_1, var_names=["mu_eye_gaze","mu_mouth_size", "mu_mov"]);

In [51]:
# pm.plot_trace(hierarchical_trace_clust_2, var_names=["mu_eye_gaze","mu_mouth_size", "mu_mov"]);

In [52]:
#  az.hdi(hierarchical_trace_clust_2,  var_names=["mu_eye_gaze","mu_mouth_size", "mu_mov"])

In [53]:
# az.plot_posterior(hierarchical_trace_clust_2, var_names=["mu_eye_gaze","mu_mouth_size", "mu_mov"], ref_val=0)

In [54]:
# n_dyads = len(data_clust3.Dyad_no_Gaze_nose_dist.unique())
# dyad_idx = np.array(data_clust3.Dyad_no_Gaze_nose_dist.str.split("_",n=1, expand = True)[1],dtype='int64')-1

# with pm.Model() as hierarchical_model_clust_3:
#     # Hyperpriors for group nodes
#     mu_intercept = pm.Normal("mu_intercept", mu=0.0, sigma=100)
#     sigma_intercept = pm.HalfNormal("sigma_intercept", 5.0)
#     mu_eye_gaze = pm.Normal("mu_eye_contact", mu=0.0, sigma=100)
#     sigma_eye_gaze = pm.HalfNormal("sigma_eye_contact", 5.0)    
  
#     mu_mouth_size = pm.Normal("mu_smiling_behavior", mu=0.0, sigma=100)
#     sigma_mouth_size = pm.HalfNormal("sigma_smiling_behavior", 5.0)  
    
#     mu_mov = pm.Normal("mu_body_movements", mu=0.0, sigma=100)
#     sigma_mov = pm.HalfNormal("sigma_body_movements", 5.0)     
    
    
#     # Intercept for each county, distributed around group mean mu_a
#     # Above we just set mu and sd to a fixed value while here we
#     # plug in a common group distribution for all a and b (which are
#     # vectors of length n_counties).
#     intercept = pm.Normal("intercept", mu=mu_intercept, sigma=sigma_intercept, shape=n_dyads)
#     # Intercept for each county, distributed around group mean mu_a
#     eye_gaze = pm.Normal("eye_contact", mu=mu_eye_gaze, sigma=sigma_eye_gaze, shape=n_dyads)
#     mouth_size = pm.Normal("smiling_behavior", mu=mu_mouth_size, sigma=sigma_mouth_size, shape=n_dyads)
#     mov = pm.Normal("body_movements", mu=mu_mov, sigma=sigma_mov, shape=n_dyads)
#     # Model error
#     eps = pm.HalfCauchy("eps", 5.0)
#     #pdb.set_trace()
# #     feature_est = intercept[dyad_idx]  + condition[dyad_idx] * data.condition.values +  eye_gaze[dyad_idx] * data.eye_gaze_distance_joint.values + mouth_size[dyad_idx] * data.mouth_size_joint.values + e[dyad_idx] * data.ALL.values
#     feature_est = intercept[dyad_idx]  +  eye_gaze[dyad_idx] * data_clust3.eye_gaze_distance_joint.values + mouth_size[dyad_idx] * data_clust3.mouth_size_joint.values +  mov[dyad_idx] * data_clust3.ALL_joint.values

#     # Data likelihood
#     feature_like = pm.Normal("feature_like", mu=feature_est, sigma=eps, observed=data_clust3.chan_freq_data)
# #     hierarchical_trace_clust_3 = pm.sample(samples, tune=tune, target_accept=0.99)

In [55]:
# gv = pm.model_graph.model_to_graphviz(hierarchical_model_clust_3)
# gv.render(filename=root_dir+'GLM_model')

In [56]:
# gv.render(filename=root_dir+'GLM_model.pdf')

In [57]:
# hierarchical_model_clust_3.vars

In [58]:
# with hierarchical_model_clust_1:
#     post_pred_clust_1 = pm.sample_posterior_predictive(hierarchical_trace_clust_1)
# # add posterior predictive to the InferenceData
# az.concat(hierarchical_trace_clust_1, az.from_pymc3(posterior_predictive=post_pred_clust_1), inplace=True)

In [59]:
# hierarchical_trace_clust_1_az = az.from_pymc3(hierarchical_trace_clust_1)

In [60]:
# with hierarchical_model_clust_1_null:
#     hierarchical_trace_clust_1_null = pm.load_trace(".pymc_3.trace")

In [61]:
# hierarchical_trace_clust_1_null_az_model = az.from_pymc3(hierarchical_trace_clust_1_null)

In [62]:
# az.loo(hierarchical_trace_clust_1_null_az_model,hierarchical_model_clust_1_null)

In [63]:
# with hierarchical_model_clust_1:
#     hierarchical_trace_clust_1 = pm.load_trace(".pymc_4.trace")

In [64]:
# hierarchical_trace_clust_1_az_model = az.from_pymc3(hierarchical_trace_clust_1)

In [65]:
# az.loo(hierarchical_trace_clust_1_az_model,hierarchical_model_clust_1)

In [66]:
# hierarchical_trace_clust_1_az_model.to_netcdf('hierarchical_trace_clust_1_az_model')

In [67]:
# hierarchical_trace_clust_1_null_az_model.to_netcdf('hierarchical_trace_clust_1_null_az_model')

In [68]:
# hierarchical_trace_clust_1_az_model = az.from_netcdf('hierarchical_trace_clust_1_az_model')
# hierarchical_trace_clust_1_null_az_model.to_netcdf('hierarchical_trace_clust_1_null_az_model')

In [69]:
# df_comp_loo = az.compare({"hierarchical": hierarchical_trace_clust_1_az_model, "pooled": hierarchical_trace_clust_1_null_az_model})
# df_comp_loo

In [70]:
# print(np.sum(hierarchical_trace_clust_1['mu_eye_gaze']<0)/len(hierarchical_trace_clust_1['mu_eye_gaze']))

In [71]:
# with hierarchical_model_clust_2:
#     post_pred_clust_2 = pm.sample_posterior_predictive(hierarchical_trace_clust_2)
# # add posterior predictive to the InferenceData
# az.concat(hierarchical_trace_clust_2, az.from_pymc3(posterior_predictive=post_pred_clust_2), inplace=True)